### Luigi Module

In [15]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

# Dealing with multiple outputs

dataset = pars.get('dataset')

KeyError: 'PARS'

In [2]:
import pandas as pd

In [16]:
#input_file = '../../data/stack-overflow/pandas-solutioncode-h1'
#output_file = '../../data/stack-overflow/result1.txt'
#dataset = '../../data/stack-overflow/Dataset - Pandas.csv'
#dataset = pd.read_csv(dataset, encoding='ISO-8859-1', error_bad_lines=False)

result_df = pd.read_pickle(input_file)


,Unnamed: 0,Unnamed: 0.1,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,...,PostTypeId,Score,Tags,Title,ViewCount,Code,PreprocessedCode,PreprocessedCode2,PreprocessedCode3,Solution
0,27,27,7779260.0,2.0,<p>When I try to merge two dataframes by rows ...,NaN,7,NaN,2011-10-15T08:21:17.460,4.0,...,1,25,<python><pandas>,append two data frame with pandas,25479.0,bigdata = data1.append(data2)\nException: Inde...,bigdata = data1.append(data2)\nException: Inde...,bigdata = data1.append(data2)\nException: Inde...,bigdata = data1.append(data2),NA
1,31,31,11617194.0,7.0,<p>I want to perform my own complex operations...,NaN,3,NaN,2011-10-20T14:46:14.633,108.0,...,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,195364.0,"Date,Open,High,Low,Close,Volume,Adj Close\n201...","Date,Open,High,Low,Close,Volume,Adj Close\n201...","Date,Open,High,Low,Close,Volume,Adj Close\n201...","2011-10-19,27.37,27.47,27.01,27.13,42880000,27...",NA
2,49,49,8916746.0,2.0,<p>I have a dataframe <code>df</code> in panda...,NaN,0,NaN,2012-01-18T19:41:27.017,12.0,...,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,19805.0,"df_greater_than10 = df[df[""colA""] > 10]\n","df_greater_than10 = df[df[""colA""] > 10]\n","df_greater_than10 = df[df[""colA""] > 10]\n","df_greater_than10 = df[df[""colA""] > 10]",NA
3,54,54,8997908.0,3.0,"<p>I recently came across the <a href=""http://...",NaN,16,NaN,2012-01-24T17:59:53.850,58.0,...,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,16173.0,,,,,NA
4,73,73,9557319.0,8.0,<p>I used Enthought's python distribution as a...,2013-12-04T20:55:59.357,5,NaN,2012-03-04T14:25:36.287,6.0,...,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,6585.0,,,,,NA


In [18]:
def buildTaggedDatasetDSForEvaluation(tagged_dataset_df):
    taggedDSEvalDict = dict()
    try:
        for index, row in tagged_dataset_df.iterrows():
            Id = row['AnswerId']
            if Id != 0:
                
                if Id in taggedDSEvalDict:
                    solutionList = taggedDSEvalDict[Id]
                else:
                    solutionList = list()                    
                lineList = list()
                lines = row['Solution'].split('\n')
                for line in lines:
                    if line.strip() != '':
                        pair = [line, False]
                        lineList.append(pair)
                solutionList.append(lineList)
                taggedDSEvalDict[Id]= solutionList
        
    except Exception as e:
        print(e)
    return taggedDSEvalDict


def buildResultSetForEvaluation(result_df):
    resultSetDict = dict()
    try:
        for index, row in result_df.iterrows():
            ansId = row['Id']
            solutionLinesTuple = list()                    
            predictedLines = row['Solution']
            for line in predictedLines:
                if line.strip() != '':
                    pair = [line, False]
                    solutionLinesTuple.append(pair)
            resultSetDict[ansId]= solutionLinesTuple
        
    except Exception as e:
        print(ansId, e)
    return resultSetDict

def markTheLines(taggedDSEvalDict, resultSetDict):
    for Id, pairs in resultSetDict.items():
        try:
            solutionList = taggedDSEvalDict[Id] # Error
            for respair in pairs:
                for solution in solutionList:
                    for tagpair in solution:
                        if tagpair[0].strip() == respair[0].strip():
                            tagpair[1] = True
                            respair[1] = True
        except KeyError:
            pass # Ignore it for now
    return taggedDSEvalDict, resultSetDict

In [19]:
def evaluate(result_df):
    TP = 0
    FP = 0
    taggedDSEvalDict =  buildTaggedDatasetDSForEvaluation(dataset)
    resultSetDict = buildResultSetForEvaluation(result_df)
    taggedDSEvalDict, resultSetDict = markTheLines(taggedDSEvalDict, resultSetDict)
    for Id, pairs in resultSetDict.items():
        try:
            MA = True
            for pair in pairs:
                MA = MA and pair[1]
            if MA:
                MP = False

                solutions = taggedDSEvalDict[Id]
                for solution in solutions:
                    MPS = True
                    for tagpair in solution:
                        MPS = MPS and tagpair[1]
                    MP = MP or MPS
                if MP:
                    TP = TP + 1
                else:
                    FP = FP + 1

            else:
                FP = FP + 1
        except KeyError:
            pass # Ignore it for now
            
    return float(TP)/(TP+FP)

precision = evaluate(result_df)
precision = precision * 100
print(precision)

17841294 'float' object has no attribute 'split'
0.7081625046589639


In [20]:

with open(output_file + '.txt', 'w') as f:
    f.write('%.2f' % precision)
